In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='olO5QEbyWu8txtIKuMtgttD6eUb4txQJGmbr89xNeN0F',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'ibmgenai-donotdelete-pr-rvn0zdmzlfczpe'
object_key = 'product_reviews.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)


,review_id,review_text
0,1,"The camera quality is outstanding, especially ..."
1,2,I love the battery life—it lasts me two full d...
2,3,"The screen is vibrant and sharp, but the refre..."
3,4,Performance is smooth even with multiple apps ...
4,5,The speaker quality is poor and sounds distort...
5,6,It charges super fast but the battery drains q...
6,7,"The phone feels premium in hand, and the desig..."
7,8,Face recognition works flawlessly but fingerpr...
8,9,The display is crisp and very responsive to to...
9,10,It heats up a bit during gaming but runs most ...


In [2]:
from ibm_watsonx_ai.foundation_models import ModelInference
import os, getpass
import pandas as pd
import time

# Load your DataFrame (assuming it's already available as df_1)
df = df_1.copy()
df["extracted_features"] = ""

# Set up model credentials
def get_credentials():
    return {
        "url" : "https://us-south.ml.cloud.ibm.com",
        "apikey" : getpass.getpass("Please enter your api key (hit enter): ")
    }

parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 30,
    "min_new_tokens": 1,
    "stop_sequences": [],
    "repetition_penalty": 1
}

model = ModelInference(
    #model_id = "ibm/granite-3-8b-instruct",
    model_id = "google/flan-t5-xl",
    params = parameters,
    credentials = get_credentials(),
    project_id = os.getenv('PROJECT_ID')  # Or paste it directly if needed
)

# Prompt template
prompt_template = """Extract the main product features mentioned in the customer review. Focus on technical or functional aspects like camera, battery, screen, design, etc.

Review: "The camera quality is outstanding, especially in low light conditions."
Features: camera quality

Review: "I love the battery life—it lasts me two full days without charging."
Features: battery life

Review: "The screen is vibrant and sharp, but the refresh rate could be better."
Features: screen, refresh rate

Review: "{review_text}"
Features:"""

# Run inference for each review
for i, row in df.iterrows():
    prompt = prompt_template.format(review_text=row["review_text"])
    try:
        print(f"Processing review_id={row['review_id']}...")
        response = model.generate_text(prompt=prompt, guardrails=True)
        df.at[i, "extracted_features"] = response.strip()
        time.sleep(0.5)  # To avoid hitting rate limits
    except Exception as e:
        df.at[i, "extracted_features"] = f"Error: {str(e)}"

# View results
print(df[["review_id", "review_text", "extracted_features"]])


Please enter your api key (hit enter):  ········


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'google/flan-t5-xl' is in deprecated state from 2025-06-18 until 2025-10-15. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


Processing review_id=1...
Processing review_id=2...
Processing review_id=3...
Processing review_id=4...
Processing review_id=5...
Processing review_id=6...
Processing review_id=7...
Processing review_id=8...
Processing review_id=9...
Processing review_id=10...
Processing review_id=11...
Processing review_id=12...
Processing review_id=13...
Processing review_id=14...
Processing review_id=15...
Processing review_id=16...
Processing review_id=17...
Processing review_id=18...
Processing review_id=19...
Processing review_id=20...
    review_id                                        review_text  \
0           1  The camera quality is outstanding, especially ...   
1           2  I love the battery life—it lasts me two full d...   
2           3  The screen is vibrant and sharp, but the refre...   
3           4  Performance is smooth even with multiple apps ...   
4           5  The speaker quality is poor and sounds distort...   
5           6  It charges super fast but the battery drains q